### Importing Libraries 

In [2]:
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
import os
import cv2
import matplotlib.pyplot as plt
import PIL.Image as img
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
train_dir = './Downloads/Covid_19/Covid19-dataset/train'
test_dir = './Downloads/Covid_19/Covid19-dataset/test'

In [5]:
x_train = []
for folder in os.listdir(train_dir):
    sub_path = train_dir + '/' + folder
    for image in os.listdir(sub_path):
        img_path = sub_path + '/' + image
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224,244))
        x_train.append(img)

In [6]:
x_test = []
for folder in os.listdir(test_dir):
    sub_test = test_dir + '/' + folder
    for image in os.listdir(sub_test):
        img_test = sub_test + '/' + image
        imgtest = cv2.imread(img_test)
        imgtest = cv2.resize(imgtest,(224,224))
        x_test.append(imgtest)

In [7]:
train = np.array(x_train)/255.0
test = np.array(x_test)/255.0

In [8]:
train_data = ImageDataGenerator(rescale = 1/255.0 , validation_split=0.2)
test_data = ImageDataGenerator(rescale = 1/255.0)

In [9]:
train_set = train_data.flow_from_directory(
                    train_dir,
                    subset = 'training',
                    batch_size=32,
                    target_size=(224,224),
                    class_mode = 'sparse'
)
val_set = train_data.flow_from_directory(
                    train_dir,
                    subset = 'validation',
                    batch_size=32,
                    target_size=(224,224),
                    class_mode = 'sparse'
)
test_set = test_data.flow_from_directory(
                    test_dir,
                    batch_size=32,
                    target_size=(224,224),
                    class_mode = 'sparse'
)

Found 201 images belonging to 3 classes.
Found 50 images belonging to 3 classes.
Found 66 images belonging to 3 classes.


In [10]:
y_train = train_set.classes 
y_val = val_set.classes
y_test = test_set.classes 
print(train_set.class_indices)

{'Covid': 0, 'Normal': 1, 'Viral Pneumonia': 2}


In [ ]:
plt.figure(figsize=(16,16))
for i in range(8):
    plt.subplot(2,4,i+1)
    plt.imshow(train[i])
    plt.title(y_train[i])

In [3]:
vgg = VGG19(include_top = False , input_shape = (224,224,3),weights='imagenet')
for layer in vgg.layers :
    layer.trainable = False

In [11]:
ES = tf.keras.callbacks.EarlyStopping(
    patience = 5,
    min_delta = 0.001,
    verbose = 0
)
model = tf.keras.Sequential([
    layers.RandomFlip(mode='horizontal'),
    layers.RandomRotation(factor=0.2),
    vgg,
    layers.Flatten(),
    layers.Dense(3, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
training = model.fit_generator(
    train_set,
    callbacks=[ES],
    steps_per_epoch = 7 ,
    epochs = 30,
    validation_data = val_set
)

C:\Users\User\AppData\Local\Temp\ipykernel_16640\887904915.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  training = model.fit_generator(


Epoch 1/30
7/7 [==============================] - 201s 29s/step - loss: 1.5072 - accuracy: 0.6169 - val_loss: 0.9107 - val_accuracy: 0.5400
Epoch 2/30
7/7 [==============================] - 165s 24s/step - loss: 0.6419 - accuracy: 0.7164 - val_loss: 0.7860 - val_accuracy: 0.6800
Epoch 3/30
7/7 [==============================] - 177s 25s/step - loss: 0.4641 - accuracy: 0.8060 - val_loss: 0.5229 - val_accuracy: 0.7800
Epoch 4/30
7/7 [==============================] - 165s 24s/step - loss: 0.3369 - accuracy: 0.8557 - val_loss: 0.6410 - val_accuracy: 0.7800
Epoch 5/30
7/7 [==============================] - 146s 21s/step - loss: 0.3215 - accuracy: 0.8756 - val_loss: 0.5623 - val_accuracy: 0.7000
Epoch 6/30
7/7 [==============================] - 148s 21s/step - loss: 0.2847 - accuracy: 0.8657 - val_loss: 0.4942 - val_accuracy: 0.7800
Epoch 7/30
7/7 [==============================] - 146s 21s/step - loss: 0.2011 - accuracy: 0.9453 - val_loss: 0.2894 - val_accuracy: 0.8600
Epoch 8/30
7/7 [====

In [ ]:

plt.plot(training.history['loss'], label='train loss')
plt.plot(training.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.plot(training.history['accuracy'], label='train acc')
plt.plot(training.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [ ]:
history_df = pd.DataFrame(training.history)
history_df.loc[:, ['loss', 'val_loss']].plot();

In [ ]:
pred = model.predict(test_set)
final_pred = np.argmax(pred, axis =1)